In [55]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopandas
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

import html5lib
!pip install lxml

In [62]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: JMBVGR3NI4NJVGIZQ1PGHYJJBSV1ISOJRNI10IIOSVH5IU2Z
CLIENT_SECRET:EO2MVFGWIOYUNJOJJPU31YLLF155FKS5TYHXSKMA5X1EZ0GL
0.24.1


# Toronto

In [57]:
#Toronto data
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dflist = pd.read_html(wiki_url)[0]
#Pre-processing
#drop rows with Borough='Not assigned'
df=dflist.drop(dflist.index[dflist.Borough=='Not assigned'])
df.sort_values(by=['Postal Code'], inplace=True)
df.reset_index(drop=True, inplace=True)
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
geodf=pd.read_csv('toronto_data.csv')
df.set_index('Postal Code', inplace=True)
geodf.set_index('Postal Code',  inplace=True)
toronto_df=df.merge(geodf, left_index=True, right_index=True)
toronto_df.reset_index(inplace=True)

torontocity_data = toronto_df[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto1 = torontocity_data.drop(['Postal Code'], axis=1)
print(df_toronto1.shape)
df_toronto1.head()

(39, 4)


,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [58]:
df_toronto=df_toronto1.set_index(['Borough','Latitude','Longitude']).stack().str.split(',', expand=True).stack()
df_toronto=df_toronto.unstack(-2).reset_index(-1,drop=True).reset_index()
df_toronto = df_toronto.reindex(columns=['Borough','Neighborhood','Latitude', 'Longitude'])
print(df_toronto.shape)
df_toronto.head(78)

(78, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,The Annex,43.672710,-79.405678
1,Central Toronto,North Midtown,43.672710,-79.405678
2,Central Toronto,Yorkville,43.672710,-79.405678
3,Central Toronto,Summerhill West,43.686412,-79.400049
4,Central Toronto,Rathnelly,43.686412,-79.400049
5,Central Toronto,South Hill,43.686412,-79.400049
6,Central Toronto,Forest Hill SE,43.686412,-79.400049
7,Central Toronto,Deer Park,43.686412,-79.400049
8,Central Toronto,Moore Park,43.689574,-79.383160
9,Central Toronto,Summerhill East,43.689574,-79.383160


In [59]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [86]:
#Function to have all the venue from the different neighborhoods
LIMIT=500
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [92]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Annex


KeyError: 'groups'

In [66]:
print(toronto_venues.shape)
toronto_venues.head()

(3181, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Annex,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
1,The Annex,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
2,The Annex,43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
3,The Annex,43.67271,-79.405678,Krispy Kreme Doughnut Cafe,43.674732,-79.407730,Donut Shop
4,The Annex,43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant


# New York

In [67]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as json_data:
        newyork_data = json.load(json_data)

In [68]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


df_ny=neighborhoods
print(df_ny.shape)
df_ny.head()


(306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [69]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ny['Latitude'], df_ny['Longitude'], df_ny['Borough'], df_ny['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [138]:
ny_venues = getNearbyVenues(names=df_ny['Neighborhood'],
                                   latitudes=df_ny['Latitude'],
                                   longitudes=df_ny['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [140]:
print(ny_venues.shape)
ny_venues.head()

(9972, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


# Zurich

In [73]:
wiki_z='https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich'
df = pd.read_html(wiki_z)[1]
df=pd.DataFrame(df)
df= df[df.District.str.get(0).isin(['D'])]
df.drop(['Location', 'Coat of Arms', 'BFS-Code', 'Incorporation', 'Areain km²','Population2005','Non-Swiss Citizens'],axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(20)

,District,Neighborhood
0,District 1Altstadt,Rathaus Hochschulen Lindenhof City
1,District 2,Wollishofen Leimbach Enge
2,District 3Wiedikon,Alt-Wiedikon Friesenberg Sihlfeld
3,District 4Aussersihl,Werd Langstrasse Hard
4,District 5Industriequartier,Gewerbeschule Escher Wyss
5,District 6,Unterstrass Oberstrass
6,District 7,Fluntern Hottingen Hirslanden Witikon
7,District 8Riesbach,Seefeld Mühlebach Weinegg
8,District 9,Albisrieden Altstetten
9,District 10,Höngg Wipkingen


In [74]:
#rename District name
town=['Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich','Zurich']
df['Town']=town
borough=['Altstadt', 'Enge & Wollishofen', 'Wiedikon', 'Aussersihl', 'Industriequartier', 'Unterstrass', 'Zurich Hill', 'Riesbach', 'Albisrieden', 'Höngg', 'Zürich Nord','Schwamendingen']
df['Borough']=borough
df.drop(['District'],axis=1,inplace=True)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]

df=df[cols]
df.head(12)

,Borough,Neighborhood,Town
0,Altstadt,Rathaus Hochschulen Lindenhof City,Zurich
1,Enge & Wollishofen,Wollishofen Leimbach Enge,Zurich
2,Wiedikon,Alt-Wiedikon Friesenberg Sihlfeld,Zurich
3,Aussersihl,Werd Langstrasse Hard,Zurich
4,Industriequartier,Gewerbeschule Escher Wyss,Zurich
5,Unterstrass,Unterstrass Oberstrass,Zurich
6,Zurich Hill,Fluntern Hottingen Hirslanden Witikon,Zurich
7,Riesbach,Seefeld Mühlebach Weinegg,Zurich
8,Albisrieden,Albisrieden Altstetten,Zurich
9,Höngg,Höngg Wipkingen,Zurich


In [75]:
from geopy.extra.rate_limiter import RateLimiter

locator=Nominatim(user_agent='myGeocoder')
df['Adress']=df['Borough']+ ', '+ df['Town']
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Adress'].apply(geocode)
# 3 - create longitude, latitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['Latitude', 'Longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df = df.drop(['Town', 'Adress', 'location','altitude', 'point'], axis=1)
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Altstadt,Rathaus Hochschulen Lindenhof City,47.372233,8.542329
1,Enge & Wollishofen,Wollishofen Leimbach Enge,47.351598,8.526223
2,Wiedikon,Alt-Wiedikon Friesenberg Sihlfeld,47.366267,8.510713
3,Aussersihl,Werd Langstrasse Hard,47.378714,8.521200
4,Industriequartier,Gewerbeschule Escher Wyss,47.387507,8.520627


In [76]:
df=pd.DataFrame(df)
df_zurich=df.set_index(['Borough','Latitude','Longitude']).stack().str.split(' ', expand=True).stack()
df_zurich=df_zurich.unstack(-2).reset_index(-1,drop=True).reset_index()
df_zurich= df_zurich.reindex(columns=['Borough','Neighborhood','Latitude', 'Longitude'])
print(df_zurich.shape)
df_zurich.head()

(36, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Albisrieden,Albisrieden,47.374857,8.484657
1,Albisrieden,Altstetten,47.374857,8.484657
2,Altstadt,Rathaus,47.372233,8.542329
3,Altstadt,Hochschulen,47.372233,8.542329
4,Altstadt,Lindenhof,47.372233,8.542329


In [77]:
address = 'Zurich,CH'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_zurich['Latitude'], df_zurich['Longitude'], df_zurich['Borough'], df_zurich['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)  
    
map_zurich

In [129]:
zurich_venues = getNearbyVenues(names=df_zurich['Neighborhood'],
                                   latitudes=df_zurich['Latitude'],
                                   longitudes=df_zurich['Longitude']
                                  )

Albisrieden
Altstetten
Rathaus
Hochschulen
Lindenhof
City
Werd
Langstrasse
Hard
Wollishofen
Leimbach
Enge
Höngg
Wipkingen
Gewerbeschule
Escher
Wyss
Seefeld
Mühlebach
Weinegg
Saatlen
Schwamendingen
Mitte
Hirzenbach
Unterstrass
Oberstrass
Alt-Wiedikon
Friesenberg
Sihlfeld
Fluntern
Hottingen
Hirslanden
Witikon
Affoltern
Oerlikon
Seebach


In [226]:
print(zurich_venues.shape)
zurich_venues.head()

(1472, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Albisrieden,47.374857,8.484657,Coop,47.375718,8.487185,Supermarket
1,Albisrieden,47.374857,8.484657,Sternen Albisrieden,47.375289,8.485041,Swiss Restaurant
2,Albisrieden,47.374857,8.484657,Migros,47.374872,8.485780,Grocery Store
3,Albisrieden,47.374857,8.484657,Emilia Trattoria Pizzeria,47.375923,8.488154,Trattoria/Osteria
4,Albisrieden,47.374857,8.484657,VBZ Fellenbergstrasse,47.375826,8.488087,Tram Station


In [174]:
to_venues=toronto_venues
zh_venues=zurich_venues

In [201]:
#number of venues per Neighborhood
ny_neigh = ny_venues[['Neighborhood','Venue']].groupby('Neighborhood').count()
to_neigh = to_venues[['Neighborhood','Venue']].groupby('Neighborhood').count()
zh_neigh = zh_venues[['Neighborhood','Venue']].groupby('Neighborhood').count()

ny_moy = ny_neigh[['Venue']].sum() / df_ny[['Neighborhood'][0]].count()
to_moy = to_neigh[['Venue']].sum() / df_toronto[['Neighborhood'][0]].count()
zh_moy = zh_neigh[['Venue']].sum() / df_zurich[['Neighborhood'][0]].count()

print('Venue per Neighborhood in NY:', ny_moy[0])
print('Venue per Neighborhood in TO:', to_moy[0])
print('Venue per Neighborhood in ZH:', zh_moy[0])

Venue per Neighborhood in NY: 32.588235294117645
Venue per Neighborhood in TO: 40.782051282051285
Venue per Neighborhood in ZH: 40.888888888888886


In [178]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))
print('There are {} uniques categories.'.format(len(zh_venues['Venue Category'].unique())))

There are 432 uniques categories.
There are 237 uniques categories.
There are 122 uniques categories.


In [238]:
regex = '.*(Neighborhood|Restaurant|Bar|tratoria|café|backery|bagel|food|diner|breakfast|bistro|tea|burrito|butcher|pizz|brewery|creperie|fish|place|pub|steak|gastro).*'

In [240]:
#NY
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

ny_grouped = ny_onehot.groupby('Neighborhood').sum().reset_index()
ny_grouped = ny_grouped.append(ny_grouped.drop('Neighborhood', axis=1).sum().rename('Total'))
ny_grouped.at['Total', 'Neighborhood'] = 'Total'
ny_grouped = ny_grouped.reset_index(drop=True)
ny_grouped = ny_grouped.filter(regex=regex, axis=1)
ny_grouped ['Total'] = ny_grouped.sum(axis=1)
ny_grouped.tail(5)

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Bar,Beach Bar,Beer Bar,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cocktail Bar,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hookah Bar,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Piano Bar,Polish Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Sake Bar,Salon / Barbershop,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Tiki Bar,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Total
297,Woodlawn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
298,Woodrow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
299,Woodside,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
300,Yorkville,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,36.0
301,Total,3.0,5.0,155.0,9.0,10.0,40.0,3.0,2.0,227.0,2.0,15.0,5.0,8.0,1.0,3.0,80.0,3.0,216.0,90.0,2.0,4.0,19.0,1.0,8.0,10.0,10.0,14.0,3

In [239]:
#TO
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = to_venues['Neighborhood'] 

to_grouped = to_onehot.groupby('Neighborhood').sum().reset_index()
to_grouped = to_grouped.append(to_grouped.drop('Neighborhood', axis=1).sum().rename('Total'))
to_grouped.at['Total', 'Neighborhood'] = 'Total'
to_grouped = to_grouped.reset_index(drop=True)
to_grouped = to_grouped.filter(regex= regex, axis=1)
to_grouped ['Total'] = to_grouped.sum(axis=1)
to_grouped.tail(5)

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bar,Beer Bar,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hookah Bar,Hotel Bar,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Salon / Barbershop,Seafood Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Total
73,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,The Danforth West,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
75,Toronto Dominion Centre,0.0,3.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,35.0
76,University of Toronto,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
77,Total,1.0,40.0,22.0,61.0,31.0,4.0,7.0,2.0,8.0,16.0,19.0,5.0,9.0,4.0,3.0,4.0,3.0,3.0,25.0,3.0,16.0,27.0,4.0,1.0,9.0,24.0,2.0,3.0,13.0,73.0,56.0,6.0,3.0,8.0,2.0,7.0,21.0,14.0,6.0,1.0,2.0,17.0,2.0,6.0,107.0,1.0,13.0,35.0,8.0,27.0,48.0,2.0,31.0,1.0,36.0,18.0,17.0,937.0


In [227]:
#ZH
# one hot encoding
zh_onehot = pd.get_dummies(zh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back zh dataframe
zh_onehot['Neighborhood'] = zh_venues['Neighborhood'] 

zh_grouped = zh_onehot.groupby('Neighborhood').sum().reset_index()
zh_grouped = zh_grouped.append(zh_grouped.drop('Neighborhood', axis=1).sum().rename('Total'))
zh_grouped.at['Total', 'Neighborhood'] = 'Total'
zh_grouped = zh_grouped.reset_index(drop=True)
zh_grouped = zh_grouped.filter(regex= regex, axis=1)
zh_grouped ['Total'] = zh_grouped.sum(axis=1)
zh_grouped.tail()

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Bar,Cambodian Restaurant,Chinese Restaurant,Cocktail Bar,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Gastropub,Gay Bar,Hotel Bar,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Peruvian Restaurant,Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Steakhouse,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Total
32,Wipkingen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
33,Witikon,0.0,1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,5.0,1.0,0.0,0.0,2.0,0.0,0.0,31.0
34,Wollishofen,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
35,Wyss,0.0,0.0,2.0,0.0,5.0,1.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,33.0
36,Total,3.0,8.0,19.0,3.0,94.0,3.0,3.0,35.0,3.0,8.0,9.0,16.0,3.0,11.0,11.0,5.0,59.0,17.0,3.0,3.0,14.0,11.0,10.0,3.0,3.0,60.0,3.0,11.0,3.0,5.0,7.0,82.0,32.0,3.0,3.0,22.0,3.0,6.0,597.0


In [ ]:
#Density of Food Place per Neighborhood
ny_food = ny_grouped['Total'].iloc[-1]
to_food = to_grouped['Total'].iloc[-1]
zh_food = zh_grouped['Total'].iloc[-1]

ny_moy_food = ny_food / df_ny[['Neighborhood'][0]].count()
to_moy_food = to_food / df_toronto[['Neighborhood'][0]].count()
zh_moy_food = zh_food / df_zurich[['Neighborhood'][0]].count()
print('Venue per Neighborhood in NY:', ny_moy_food)

In [229]:
# function to sort the food place in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
return row_categories_sorted.index.values[0:num_top_venues]

In [318]:
# NY top food place
num_top_venues = 10
ny_grouped2 = ny_grouped.drop(['Total'], axis=1)

for hood in ny_grouped2['Neighborhood']:
    temp = ny_grouped2[ny_grouped2['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_food_sorted = pd.DataFrame(columns=columns)
ny_food_sorted['Neighborhood'] = ny_grouped2['Neighborhood']

for ind in np.arange(ny_grouped2.shape[0]):
    ny_food_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped2.iloc[ind, :], num_top_venues)

ny_food_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Allerton,Chinese Restaurant,Fast Food Restaurant,Spanish Restaurant,Wine Bar,Egyptian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant
1,Annadale,Restaurant,Sports Bar,Wine Bar,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant
2,Arden Heights,Wine Bar,Greek Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Halal Restaurant,Jewish Restaurant
3,Arlington,American Restaurant,Wine Bar,Halal Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant
4,Arrochar,Italian Restaurant,Polish Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Wine Bar,Gluten-free Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant


In [319]:
#ny top 10 per Neighborhood
ny_food_sorted_neig = ny_food_sorted.drop(ny_food_sorted.tail(1).index)
ny_food_sorted_neig.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
296,Woodhaven,Restaurant,Latin American Restaurant,Mexican Restaurant,Hookah Bar,Thai Restaurant,Arepa Restaurant,Dive Bar,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Gluten-free Restaurant,Filipino Restaurant,French Restaurant,English Restaurant,Gastropub
297,Woodlawn,Italian Restaurant,Indian Restaurant,Bar,Wine Bar,Greek Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar,German Restaurant,Gluten-free Restaurant
298,Woodrow,Juice Bar,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,Italian Restaurant,Indonesian Restaurant,Indian Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gastropub,Gay Bar
299,Woodside,Filipino Restaurant,Thai Restaurant,Latin American Restaurant,Bar,American Restaurant,Chinese Restaurant,Japanese Restaurant,Restaurant,Fast Food Restaurant,Salon / Barbershop,Empanada Restaurant,Himalayan Restaurant,Arepa Restaurant,Asian Restaurant,Tibetan Restaurant
300,Yorkville,Italian Restaurant,Bar,Sushi Restaurant,Japanese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Wine Bar,Latin American Restaurant,German Restaurant,French Restaurant,New American Restaurant,Peruvian Restaurant,Salon / Barbershop,Indian Restaurant


In [320]:
#ny top 10
ny = ny_food_sorted.iloc[[-1]]
ny = ny.reset_index(drop=True)
ny.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Total,Italian Restaurant,Bar,Chinese Restaurant,Mexican Restaurant,American Restaurant,Sushi Restaurant,Fast Food Restaurant,Japanese Restaurant,Cocktail Bar,Thai Restaurant,Restaurant,Latin American Restaurant,Caribbean Restaurant,Seafood Restaurant,Wine Bar


In [321]:
# to top food place
num_top_venues = 10
to_grouped2 = to_grouped.drop(['Total'], axis=1)

for hood in to_grouped2['Neighborhood']:
    temp = to_grouped2[to_grouped2['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
to_food_sorted = pd.DataFrame(columns=columns)
to_food_sorted['Neighborhood'] = to_grouped2['Neighborhood']

for ind in np.arange(to_grouped2.shape[0]):
    to_food_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped2.iloc[ind, :], num_top_venues)

to_food_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adelaide,Restaurant,Thai Restaurant,American Restaurant,Sushi Restaurant,Steakhouse,Modern European Restaurant,New American Restaurant,Fast Food Restaurant,Gluten-free Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Colombian Restaurant,Mediterranean Restaurant,Gastropub
1,Bathurst Quay,Bar,Wine Bar,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,Gay Bar,Gastropub,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Cuban Restaurant
2,Cabbagetown,Restaurant,Italian Restaurant,Chinese Restaurant,Indian Restaurant,Thai Restaurant,Taiwanese Restaurant,Japanese Restaurant,Gastropub,Caribbean Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,Filipino Restaurant,Doner Restaurant
3,Chinatown,Mexican Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Caribbean Restaurant,Doner Restaurant,Filipino Restaurant,Comfort Food Restaurant,Cocktail Bar,Wine Bar,Japanese Restaurant,Belgian Restaurant,Beer Bar,Falafel Restaurant,Gastropub
4,Deer Park,Sports Bar,American Restaurant,Vietnamese Restaurant,Sushi Restaurant,Restaurant,Filipino Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Wine Bar,French Restaurant,Cuban Restaurant,Gastropub,Gay Bar


In [322]:
#to top 10 per Neighborhood
to_food_sorted_neig = to_food_sorted.drop(to_food_sorted.tail(1).index)
to_food_sorted_neig.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
72,The Annex,Indian Restaurant,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Cuban Restaurant,Gluten-free Restaurant,German Restaurant,Gay Bar,Gastropub,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Doner Restaurant
73,The Beaches,Wine Bar,Hotel Bar,Greek Restaurant,Gluten-free Restaurant,German Restaurant,Gay Bar,Gastropub,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Doner Restaurant,Cuban Restaurant
74,The Danforth West,Greek Restaurant,Italian Restaurant,Restaurant,Indian Restaurant,American Restaurant,Japanese Restaurant,Juice Bar,Caribbean Restaurant,Filipino Restaurant,Falafel Restaurant,Fast Food Restaurant,Gastropub,French Restaurant,Eastern European Restaurant,Gay Bar
75,Toronto Dominion Centre,Restaurant,American Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Steakhouse,Beer Bar,Wine Bar,New American Restaurant,Gluten-free Restaurant,Greek Restaurant,French Restaurant,Fast Food Restaurant
76,University of Toronto,Restaurant,Italian Restaurant,Bar,Japanese Restaurant,Comfort Food Restaurant,Beer Bar,Sushi Restaurant,French Restaurant,Chinese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Gay Bar,Gastropub,Eastern European Restaurant


In [323]:
#to top 10
to = to_food_sorted.iloc[[-1]]
to = to.reset_index(drop=True)
to.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Total,Restaurant,Italian Restaurant,Bar,Japanese Restaurant,Sushi Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Beer Bar,Thai Restaurant,Steakhouse,Gastropub,Fast Food Restaurant,Greek Restaurant,Asian Restaurant


In [324]:
# zh top food place
num_top_venues = 10
zh_grouped2 = zh_grouped.drop(['Total'], axis=1)


for hood in zh_grouped2['Neighborhood']:
    temp = zh_grouped2[zh_grouped2['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

    
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zh_food_sorted = pd.DataFrame(columns=columns)
zh_food_sorted['Neighborhood'] = zh_grouped2['Neighborhood']

for ind in np.arange(zh_grouped2.shape[0]):
    zh_food_sorted.iloc[ind, 1:] = return_most_common_venues(zh_grouped2.iloc[ind, :], num_top_venues)

zh_food_sorted.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
32,Wipkingen,Italian Restaurant,Mexican Restaurant,Steakhouse,Fast Food Restaurant,Falafel Restaurant,Indian Restaurant,Hotel Bar,Gay Bar,Gastropub,French Restaurant,Wine Bar,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Cambodian Restaurant
33,Witikon,Swiss Restaurant,Bar,Cocktail Bar,Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,French Restaurant,Gay Bar,Japanese Restaurant,Mediterranean Restaurant,Hotel Bar,Spanish Restaurant,Sushi Restaurant,Thai Restaurant,Argentinian Restaurant
34,Wollishofen,American Restaurant,Restaurant,Falafel Restaurant,Indian Restaurant,Hotel Bar,Gay Bar,Gastropub,French Restaurant,Fast Food Restaurant,Eastern European Restaurant,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Cambodian Restaurant,Bar
35,Wyss,Bar,Restaurant,Italian Restaurant,Swiss Restaurant,Mexican Restaurant,Falafel Restaurant,Asian Restaurant,Thai Restaurant,Fast Food Restaurant,Cambodian Restaurant,Japanese Restaurant,Southern / Soul Food Restaurant,Gastropub,Cocktail Bar,Modern European Restaurant
36,Total,Bar,Swiss Restaurant,Restaurant,Italian Restaurant,Cocktail Bar,Thai Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Japanese Restaurant,French Restaurant,Mediterranean Restaurant,Gay Bar,Hotel Bar,Mexican Restaurant,Spanish Restaurant


In [325]:
#ZH top 10 per Neighborhood
zh_food_sorted_neig = zh_food_sorted.drop(zh_food_sorted.tail(1).index)
zh_food_sorted_neig.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
31,Werd,Bar,Italian Restaurant,Thai Restaurant,Swiss Restaurant,Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Wine Bar,Gay Bar,Indian Restaurant,Vietnamese Restaurant,Mediterranean Restaurant,Chinese Restaurant,Peruvian Restaurant
32,Wipkingen,Italian Restaurant,Mexican Restaurant,Steakhouse,Fast Food Restaurant,Falafel Restaurant,Indian Restaurant,Hotel Bar,Gay Bar,Gastropub,French Restaurant,Wine Bar,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Cambodian Restaurant
33,Witikon,Swiss Restaurant,Bar,Cocktail Bar,Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,French Restaurant,Gay Bar,Japanese Restaurant,Mediterranean Restaurant,Hotel Bar,Spanish Restaurant,Sushi Restaurant,Thai Restaurant,Argentinian Restaurant
34,Wollishofen,American Restaurant,Restaurant,Falafel Restaurant,Indian Restaurant,Hotel Bar,Gay Bar,Gastropub,French Restaurant,Fast Food Restaurant,Eastern European Restaurant,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Cambodian Restaurant,Bar
35,Wyss,Bar,Restaurant,Italian Restaurant,Swiss Restaurant,Mexican Restaurant,Falafel Restaurant,Asian Restaurant,Thai Restaurant,Fast Food Restaurant,Cambodian Restaurant,Japanese Restaurant,Southern / Soul Food Restaurant,Gastropub,Cocktail Bar,Modern European Restaurant


In [326]:
#zh top 10
zh = zh_food_sorted.iloc[[-1]]
zh = zh.reset_index(drop=True)
zh.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Total,Bar,Swiss Restaurant,Restaurant,Italian Restaurant,Cocktail Bar,Thai Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Japanese Restaurant,French Restaurant,Mediterranean Restaurant,Gay Bar,Hotel Bar,Mexican Restaurant,Spanish Restaurant


In [327]:
#append
all = ny.append(to)
all = all.append(zh)
all = all.drop(['Neighborhood'], axis=1)
all = all.reset_index(drop=True)
all.head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Italian Restaurant,Bar,Chinese Restaurant,Mexican Restaurant,American Restaurant,Sushi Restaurant,Fast Food Restaurant,Japanese Restaurant,Cocktail Bar,Thai Restaurant,Restaurant,Latin American Restaurant,Caribbean Restaurant,Seafood Restaurant,Wine Bar
1,Restaurant,Italian Restaurant,Bar,Japanese Restaurant,Sushi Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Beer Bar,Thai Restaurant,Steakhouse,Gastropub,Fast Food Restaurant,Greek Restaurant,Asian Restaurant
2,Bar,Swiss Restaurant,Restaurant,Italian Restaurant,Cocktail Bar,Thai Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Japanese Restaurant,French Restaurant,Mediterranean Restaurant,Gay Bar,Hotel Bar,Mexican Restaurant,Spanish Restaurant


In [328]:
data = {'City': ['NY',  'Toronto',  'Zurich'],
        'N. of Neighborhood': [df_ny[['Neighborhood'][0]].count(), df_toronto[['Neighborhood'][0]].count(),df_zurich[['Neighborhood'][0]].count()],
        'Density of Venue': [round(ny_moy[0]),round(to_moy[0]),round(zh_moy[0])],
        'Density of Food Place':[round(ny_moy_food),round(to_moy_food),round(zh_moy_food)],
        'Variety of Food Place':[len(ny_grouped.columns) - 2,len(to_grouped.columns) - 2,len(zh_grouped.columns) - 2]
       }
df_nb_neigh = pd.DataFrame(data,columns=['City','N. of Neighborhood','Density of Venue','Density of Food Place','Variety of Food Place'])
df_nb_neigh = pd.concat([df_nb_neigh, all],axis =1)
df_nb_neigh.head()

,City,N. of Neighborhood,Density of Venue,Density of Food Place,Variety of Food Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,NY,306,33.0,10.0,109,Italian Restaurant,Bar,Chinese Restaurant,Mexican Restaurant,American Restaurant,Sushi Restaurant,Fast Food Restaurant,Japanese Restaurant,Cocktail Bar,Thai Restaurant,Restaurant,Latin American Restaurant,Caribbean Restaurant,Seafood Restaurant,Wine Bar
1,Toronto,78,41.0,12.0,57,Restaurant,Italian Restaurant,Bar,Japanese Restaurant,Sushi Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Beer Bar,Thai Restaurant,Steakhouse,Gastropub,Fast Food Restaurant,Greek Restaurant,Asian Restaurant
2,Zurich,36,41.0,17.0,38,Bar,Swiss Restaurant,Restaurant,Italian Restaurant,Cocktail Bar,Thai Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Japanese Restaurant,French Restaurant,Mediterranean Restaurant,Gay Bar,Hotel Bar,Mexican Restaurant,Spanish Restaurant


## Report on the analysis

#### 1. Introduction (discuss the business problem and who would be interested in this project)
NY, Toronto and Zurich share the common specificity of being the financial capitals of their country but not the administrative one. NY and Toronto are both North American cities with several millions of inhabitants and are therefore expected to share similar features regarding the number and types of venue. On the other hand, Zurich is a European city, significantly smaller and most likely much less cosmopolitan than NY or Toronto.

As imagined for our case study, an exploration analysis of the venues (for example in terms of types and density) can be very useful in order to get the big picture over the type economical terrain of cities. The advantage of this analysis is to rapidly obtain facts and figures that can support a business case, for instance for opening a restaurant. The advantage is that the data are available and can quickly be updated (you could even look at the evolution over time). Facts and figures are a pre requisite in order to convince banks and investors and are the start of every good business case.

__Business Problem:__ to gain a rapid basic market understanding of geographical areas (NY, Toronto, Zurich), based on an exploratory analysis of venues, in order to draw first conclusions based on facts and figures regarding the implementation of a new venue type in an area (indian restaurant).

__Stakeholders:__ futur owner of the new venue and investors in the new venue

#### 2. Data (describe the data that will be used to solve the problem and the source of the data)
Luckily, some data were already available from the previsous lab of this module:

- New York: Data are available as a json file dataset from Week 2
- Toronto: as seen during Week 3 of this module, the data can be downloaded out of a wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
- The Data for Zurich had to be search online and were found to be available on wikipedia too

The data had then to be pre-processed in order to obtain comparable dataset: the data were preprocessed into dataframes with 4 columns: Borough, Neighborhood, Latitude and Longitude.

#### 3. Methodology (discuss and describe the exploratory data analysis, inferential statistical testing, if any, and what machine learnings were used and why)
We followed standard method for Data Science analysis:

1. __Business understanding & analytical approach:__ we started by refining the problem to be solved and to think what could be the possible output of this analysis. One pre-requisite was to make sure that we had enough understanding of the business. This was ensured by the previsous labs on the module, that gave us sufficient insight to feel comfortable starting in that direction.
2. __Data requirement, collection, understanding and preparation:__ we had to obtain comparable datasets for each city, with the required quality and required informations. The first step was to determined if the dataset for each city was plausible and correct or not. Out of the previous labs, we knew that the dataset for NY and Toronto were correct and of good quality. Regarding the dataset for Zurich, we used our own knowledge of the city to assess the quality of the data. The format and size of the data was regularly checked by the use of the 'head' and 'shape' function in order to make sure that data were of good quality, size and format. Necessary adjustments were done when needed. The data had then to be pre-processed in order to obtain comparable dataset: the data were preprocessed into dataframes with 4 columns: Borough, Neighborhood, Latitude and Longitude. Once the data were prepared, the exploratory analysis could start: using Foursquare, results are easily accessible.
3. __Modelling and Evalutation:__ After data are prepared, one need to think of how can the data be used and visualised, and determine what additional visualisations could help us answer our business problem. First we explored the venues available within a defined radius around each neighborhoods. In order to compare the three cities, the radius was kept constant for each cities. Several radius were tried and it was decided that 500 was an appropriate radius, in order to be able to get all the venues within this radius without limitation of the Foursquare account. The previous step gave us the total number of venues available around all the neighborhoods and allowed for first comparison between the cities, data were displayed in tabular form (Dataframe). In a second phase, we called for the most common venues in each neighborhoods. Due to the important number of data, it was chosen to focus on the top 5 (frequency wise) and to display the data as histograms, in order to allow fast comparison. As As for any data analysis, the whole process of modeling and evaluation was highly iterative and the next step was always influenced by the results that we acquiring, sometimes confirming our hypothesis and sometimes denying it. Several lyouts were thought through before the final layout was decided.

#### 4. Results section where you discuss the results.
The results that we obtained from the analysis were, as very often, not always confirming our hypothesis. Our main results are the following:

1. Number of neighborhoods: we looked at the number of neighborhoods per city (siehe Dataframe xy). The city with the highest number of venue is NY, followed by Toronto and then by Zurich.
2. Density of venues: we calculated what is the density of venues per city and compared them between the three cities
3. Diversity: we look at the number of different type of venues per city, NY has the highest one.
4. Favorite Type: we looked at the favorite type of venue in each city. Difference can be seen.


####  5. Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
1. Number of neighborhoods:
    - we expected a correlation between the number of inhabitants and the number of neighborhoods.
    - This was confirmed by the results: the bigger the city, the higher the numbers of neighborhoods.
2. Density of venues:
    - we had expected a correlation between the size of a city and the density of venues per neighborhoods.
    - Surprisingly, the density and the number of inhabitants were not correlated. Zurich has a higher density of venues per neighborhood.
3. Diversity (number of different type of venues): we expected bigger city to have a higher diversity as expected bigger in bigger city (more cultural diversity?) fourth: top 5 venues type -> histogramm?

Critics:

- is the database for Zurich, NY and Toronto on Foursquare comparable? we know that some application are more popular in some places are other (for instance Whatsapp very popular in Europe but not in the US)
- is a neighborhood in Zurich equivalent to one in NY? number of inhabitants per neighborhood within a radius was here consired to be comparable, is it?
- a high number of indian restanrant is a sign that the population likes indian, but what is the tendency of time? going down or up? trends are more important than fix numbers / how can one asses the effect of concurrence?

Potential:
- Analyse the changes overtime
- get the premium version to make sure all the data where extracted and see more features (price, etc.)

#### 6. Conclusion section where you conclude the report.
Interesting example to see how data are readily available online. Number and types of analysis are endless Longest is the pre processing of the data and require knowledge of the business in order to provide relevant datas (and clean them)